In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

In [8]:
# Load YOLO model
model = YOLO("yolov8n.pt")  
# Load video
video_path = "newbuslane.mp4"
cap = cv2.VideoCapture(video_path)

# Get video dimensions
frame_width = int(cap.get(3))  
frame_height = int(cap.get(4))  

# Detect non-black area (road) dynamically
def get_road_region(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)  # Detect non-black pixels
    x, y, w, h = cv2.boundingRect(thresh)  # Get bounding box of road area
    return (x, y, x + w, y + h)  # (x1, y1, x2, y2)

# Process first frame to find road region
ret, frame = cap.read()
road_x1, road_y1, road_x2, road_y2 = get_road_region(frame)

# Define bus lane inside detected road area
bus_lane_x1 = int(road_x1 + 0.10 * (road_x2 - road_x1))  
bus_lane_x2 = int(road_x1 + 0.90 * (road_x2 - road_x1))  
bus_lane_y1 = road_y1  
bus_lane_y2 = road_y2  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    # Draw detected road boundary
    cv2.rectangle(frame, (road_x1, road_y1), (road_x2, road_y2), (255, 255, 255), 2)

    # Draw bus lane inside detected road
    cv2.rectangle(frame, (bus_lane_x1, bus_lane_y1), (bus_lane_x2, bus_lane_y2), (0, 255, 0), 2)
    cv2.putText(frame, "Bus Lane", (bus_lane_x1, bus_lane_y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

   
    for r in results:
         for box in r.boxes:
            cls = int(box.cls[0])
            label = r.names[cls]

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            width = x2 - x1
            height = y2 - y1

            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Check if the vehicle is inside the bus lane
            if bus_lane_x1 < center_x < bus_lane_x2 and bus_lane_y1 < center_y < bus_lane_y2:
                if label in ["bus"]:  # Authorized vehicles
                    color = (0, 255, 0)  # Green
                    text = f"Unauthorized: {label}"
                else:  # Unauthorized vehicles
                    color = (0, 0, 255)  # Red
                    text = f"Unauthorized: {label}"

                # Draw the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, text, (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    cv2.imshow("Bus Lane Monitoring", frame)
 
    if cv2.waitKey(1) & 0xFF == ord("q"):  
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 13 persons, 9 cars, 2 motorcycles, 1 bus, 6 trucks, 2 umbrellas, 100.5ms
Speed: 6.4ms preprocess, 100.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 7 cars, 1 motorcycle, 2 buss, 6 trucks, 3 umbrellas, 86.8ms
Speed: 2.0ms preprocess, 86.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 7 cars, 1 motorcycle, 4 buss, 5 trucks, 3 umbrellas, 75.1ms
Speed: 2.1ms preprocess, 75.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 5 cars, 3 buss, 4 trucks, 3 umbrellas, 61.6ms
Speed: 1.6ms preprocess, 61.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 6 cars, 3 buss, 4 trucks, 4 umbrellas, 63.9ms
Speed: 2.0ms preprocess, 63.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 6 cars, 1 motorcycle, 3 buss, 3 trucks, 4 umbrellas, 62.0ms
Speed: 1.7ms preprocess, 62.0ms infer